# Convertir enlances a txt

### instalaciones

In [11]:
!pip install pandas tqdm beautifulsoup4 -q
!pip install pandarallel ipywidgets -q


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### imports e inicializacion de multhilo para panda

In [22]:
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import random
from pprint import pprint
from concurrent.futures import ProcessPoolExecutor, as_completed
import os
from pandarallel import pandarallel
import re
from urllib.parse import urlparse
import requests

pandarallel.initialize(progress_bar=True, nb_workers=os.cpu_count()-1)

# random.seed(42)

INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


### Coger los csv generados

In [2]:
print([*range(1,7)])
# Cargar el dataframe
df = pd.concat([pd.read_csv(f"resultados_scraper/resultados_crawl_paginas_chunk_{X}_claude.csv") for X in range(1,7)], ignore_index=True)
mask = ~df['url'].str.endswith(('html', 'odt'))
df = df[mask].reset_index(drop=True)
print(len(df))


[1, 2, 3, 4, 5, 6]
2870


### Generacion de la columna resultado

In [3]:
# Crear la columna donde ira el parseo
df["parseado"] = df["contenido"]
df.head(2)

,url,profundidad,tipo,contenido,parseado
0,https://www.cabildodelanzarote.com/mapa-web/,0,paginas,"<body class=""controls-visible blue yui3-skin-s...","<body class=""controls-visible blue yui3-skin-s..."
1,https://www.cabildodelanzarote.com/turismo-lan...,1,paginas,"<body class=""controls-visible blue yui3-skin-s...","<body class=""controls-visible blue yui3-skin-s..."


### eliminar elementos html innecesarios, eliminar hrefs innecesarios, y obtener el texto del body

In [ ]:
# Buscando elemento html y eliminarlo, y extraer el texto, con multihilo

elementos = [
    {"name":"div","id":"list-menu"},
    {"name":"footer","id":"footer"},
    {"name":"footer","class":"footer-copyright"},
    {"name":"div","id":"heading"},
    {"name":"header","id":"header"},
    {"name":"script"},
    {"name":"div", "id":"navigation-header"},
    {"name":"div", "id":"dialog-search"},
    {"name":"div", "id":"navigation-dropdown"},
    {"name":"nav", "id":"breadcrumbs"},
    {"name":"nav", "id":"fixed-menubar2"},
    {"name":"nav", "aria-label":"breadcrumb"},
    {"name":"div", "id":"list-menu secondary"},
    {"name":"div", "class":"list-menu secondary"},
    {"name":"div", "class":"dropdown"},
    {"name":"div", "class":"asset-search-portlet"},
    {"name":"div", "class":"dexportables"},
    {"name":"div", "class":"js-cookie-consent"},
    {"name":"div", "class":"cc_css_reboot"},
    {"name":"div", "class":"journal-template-error"},
    {"name":"ul", "id":"layout-1"},
    {"name":"div", "class":"menu2"},
    {"name":"div", "class":"form-group"},
    {"name":"aside", "class":"front-login"},
    {"name":"a", "class":"back-to-top"},
    {"name":"a", "class":"image-wrapper"},
    {"name":"nav", "class":"banners-menu"},
]

def limpia_boilerplate(html, elementos):
    from bs4 import BeautifulSoup # necesario, si no, falla

    soup = BeautifulSoup(html, "html.parser")

    for a in soup.find_all("a"):
        href = a.get("href", "").strip()
        text = a.get_text(strip=True)

        # Solo convertir si hay href y texto
        if ("api.whatsapp" in href) or ("facebook" in href) or ("twitter" in href):
            md = ""
        elif href and text:
            md = f"[{text}]({href})"
        elif href:
            md = f"<{href}>"
        else:
            md = text or ""
        a.string = md


    for spec in elementos:
        for tag in soup.find_all(**spec):
            tag.decompose()
    return soup.get_text()


# Aplicas una sola vez
df["parseado"] = df["parseado"].parallel_apply(
    limpia_boilerplate,
    args=(elementos,)
)

### ajustes a las strings generadas

In [ ]:
# Quitar los espacios extras
df["parseado"] = df["parseado"].str.replace("Cerrar Buscar Buscar","",)
df["parseado"] = df["parseado"].str.replace(" +"," ",regex=True)
df["parseado"] = df["parseado"].str.replace("\n+","\n",regex=True)

### Generacion del texto final, el contenido + la fuente de este

In [6]:
df = (
    df
    .assign(
        url      = lambda d: d["url"].astype(str),
        parseado = lambda d: d["parseado"].astype(str),
    )
    .assign(
        parseado = lambda d: (
            # prefijo = "fuente..." + url
            ("fuente de la siguiente información: " + d["url"])
            # primero lo juntamos con el contenido
            .str.cat(d["parseado"], sep="\n")
            # luego, al resultado le añadimos de nuevo la fuente
            .str.cat("fuente de la anterior información: " + d["url"], sep=" \n")
        )
    )
)

### Guardarlo

In [11]:
# Guardarlo
df.to_csv("resultados_scraper/resultados_crawl_paginas_todas_claude.csv", index_label="id")

### Cargarlo

In [37]:
# Cargarlo
df = pd.read_csv("resultados_scraper/resultados_crawl_paginas_todas_claude.csv",index_col="id")

### Comprobaciones

In [39]:
# Mostrar uno aleatorio
pprint(df.loc[random.randint(0,len(df)-1)].parseado)

('fuente de la siguiente información: '
 'https://www.cabildodelanzarote.com/-/el-cabildo-de-lanzarote-abre-la-inscripción-para-la-36ª-feria-de-artesanía\n'
 ' Noticias El Cabildo de Lanzarote abre la inscripción para la 36ª Feria de '
 'Artesanía Lanzarote, 04 abril 2025 Los artesanos interesados pueden '
 'presentar sus solicitudes hasta el 2 de mayo El Cabildo de Lanzarote, a '
 'través de sus áreas de Cultura y Artesanía, han anunciado la apertura del '
 'plazo de inscripción para participar en la 36ª Feria de Artesanía de '
 'Lanzarote, que se celebrará del 10 al 15 de septiembre en Mancha Blanca, '
 'Tinajo. Este evento, considerado uno de los más importantes del sector en '
 'Canarias, tiene como objetivo promover, visibilizar y preservar la riqueza '
 'artesanal de la isla, ofreciendo a los artesanos y artesanas una plataforma '
 'única para exhibir y comercializar sus creaciones. La consejera de Artesanía '
 'del Cabildo, Aroa Revelo, ha remarcado la relevancia de esta cita an

In [ ]:
# Mostrar una url concreta

# url = "https://www.cabildodelanzarote.com/deportes/enlaces"
url = "https://www.cabildodelanzarote.com/-/el-cabildo-pone-en-marcha-actividades-para-acercar-la-literatura-a-la-población-infantil-de-lanzarote"
# url = "https://transparencia.cabildodelanzarote.com/t/p/730-1018-organigrama-de-la-entidad-2023-2027"

# buscar uno en concreto
index = df[df["url"]==url].index[0]

pprint(df.loc[index].parseado,)

('fuente de la siguiente información: '
 'https://www.cabildodelanzarote.com/-/el-cabildo-pone-en-marcha-actividades-para-acercar-la-literatura-a-la-población-infantil-de-lanzarote\n'
 ' Noticias El Cabildo pone en marcha actividades para acercar la literatura a '
 'la población infantil de Lanzarote Lanzarote, 11 julio 2023 Durante los '
 'meses de julio y agosto, la Biblioteca Insular oferta un servicio de '
 'ludoteca infantil que promoverá semanas temáticas centradas en la '
 'naturaleza, el arte, la creatividad o la vida saludable El Cabildo de '
 'Lanzarote ha puesto en marcha el proyecto ‘Vive tu propio verano’ en la '
 'Biblioteca Insular, una iniciativa que busca acercar la literatura a los '
 'niños y niñas de la Isla, así como favorecer la conciliación familiar y '
 'laboral.\xa0 La consejera de Educación, Ascensión Toledo, quien visitó '
 'durante esta semana a los participantes de este servicio de ludoteca '
 'infantil, explicó que las actividades programadas se llevarán a

### Generacion del dataset de las paginas

In [11]:
df.columns

Index(['url', 'profundidad', 'tipo', 'contenido', 'parseado'], dtype='object')

In [16]:
# Generacion del csv limpio
df.drop(columns=["profundidad","tipo","contenido"]).to_csv("resultados_scraper/clean_paginas.csv")

In [17]:
# Generacion de los txt (uno por pagina)

# 1) Directorio donde guardaremos los .txt
output_dir = "salida_txt"
os.makedirs(output_dir, exist_ok=True)

# Función auxiliar para sanear nombres de archivo
def safe_filename(s):
    # elimina todo lo que no sea letra, número, guión o guión bajo
    return re.sub(r"[^\w\-]", "_", s)[:100]

# 2) Iteramos fila a fila
for idx, row in df.iterrows():
    # ejemplo de nombre: si tienes un campo 'id' o 'url' lo puedes usar:
    # base = str(row.get("id", idx))
    # o derivarlo de la URL:
    # base = safe_filename(row["url"])
    base = str(idx)  # aquí simplemente usamos el índice

    filename = f"{base}.txt"
    filepath = os.path.join(output_dir, filename)

    # 3) Escribimos el contenido de 'parseado'
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(row["parseado"])

print(f"Se han creado {len(df)} archivos en '{output_dir}/'.")

Se han creado 2870 archivos en 'salida_txt/'.


### Generacion del dataset de los pdfs

In [21]:
print([*range(1,4)])
# Cargar el dataframe
df = pd.concat([pd.read_csv(f"resultados_scraper/resultados_crawl_pdf_chunk_{X}_claude.csv") for X in range(1,4)], ignore_index=True)
df = df[mask].reset_index(drop=True)
print(len(df))

[1, 2, 3]
1120


In [36]:

# 2) Crea el directorio de salida
output_dir = "salida_pdf"
os.makedirs(output_dir, exist_ok=True)

# 3) Función para sanear y obtener nombre de fichero a partir de la URL
def get_pdf_filename(url, default_idx):
    """
    Extrae el nombre del PDF de la URL o genera uno genérico si falla.
    """
    path = urlparse(url).path            # p.ej. "/docs/archivo.pdf"
    name = os.path.basename(path)        # "archivo.pdf"
    if name.lower().endswith(".pdf") and len(name) > 4:
        return name
    # fallback
    return f"{default_idx}.pdf"

# 4) Itera y descarga
for idx, row in tqdm(df.iterrows()):
    try:
        resp = requests.get(row["url"], timeout=60, verify=False)
        resp.raise_for_status()  # lanza error si el status != 200
    except Exception as e:
        print(f"[ERROR] fila {idx} — no puede descargar {row["url"]}: {e}")
        continue

    filename = get_pdf_filename(row["url"], idx)
    filepath = os.path.join(output_dir, filename)

    # Si quieres evitar re-descargar:
    if os.path.exists(filepath):
        print(f"[SKIP] {filename} ya existe.")
        continue

    # Escribe el contenido binario
    with open(filepath, "wb") as f:
        f.write(resp.content)
    print(f"[OK] guardado: {filename}")


0it [00:00, ?it/s]c:\Users\danie\Desktop\scraper_cabildo\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'transparencia.cabildodelanzarote.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
1it [00:00,  2.73it/s]

[OK] guardado: 1744374198Informe Estado Deuda Cabildo de Lanzarote a 31.12.24 (2) .pdf


c:\Users\danie\Desktop\scraper_cabildo\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'transparencia.cabildodelanzarote.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
2it [00:01,  1.06s/it]

[OK] guardado: 166695800120200622_Catálogos del Patrimonio Histórico Artístico.pdf


c:\Users\danie\Desktop\scraper_cabildo\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'transparencia.cabildodelanzarote.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
3it [00:02,  1.23it/s]

...

[OK] guardado: 165356173539.pdf


c:\Users\danie\Desktop\scraper_cabildo\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'transparencia.cabildodelanzarote.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
1119it [08:40,  1.92it/s]

[SKIP] 165356173539.pdf ya existe.


c:\Users\danie\Desktop\scraper_cabildo\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'transparencia.cabildodelanzarote.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
1120it [08:41,  2.15it/s]

[SKIP] 165356159037.pdf ya existe.
